# Climate Data Store API

This notebook contains the research done on [Copernicus](https://cds.climate.copernicus.eu/)

### Setup

Follow the guide found [here](https://cds.climate.copernicus.eu/how-to-api)

(WIP) For the sake of simplicity, the necessary _url_ and _key_ be stored in the `.env` file 

Install dependencies
```bash
pip install cdsapi python-dotenv
```

In [5]:
import cdsapi
from dotenv import load_dotenv
import os

load_dotenv()

url = os.getenv('CDS_URL')
key = os.getenv('CDS_TOKEN')

# client = cdsapi.Client(url=url, key=url)
client = cdsapi.Client()

2024-10-03 16:57:49,917 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-03 16:57:49,917 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-10-03 16:57:49,918 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-10-03 16:57:49,918 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-10-03 16:57:49,919 WARNING [2024-06-16T


Once the CDS API client is installed, it can be used to request data from the datasets listed in the CDS, ADS, ECDS and CEMS Early Warning DS catalogues.

One must agree to the Terms of Use of a dataset before downloading any data out of it. This step must be done manually from the dataset page (at the bottom of the download form).

At the bottom of each dataset download form, press the "Show API request code" button to display a Python code snippet that shows the required fields for the request. Then simply use the `DatasetRequest` class' methods to build the request.

## Data Download

The following code creates the request to send using Copernicus

In [3]:
class DatasetRequest:
    def __init__(self):
        self.request = {}

    def __getattr__(self, name):
        if name.startswith("set_"):
            field = name[4:]

            def setter(value):
                self.request[field] = value
                return self  # Allow method chaining

            return setter
        else:
            raise AttributeError(f"'{type(self).__name__}' object has no attribute '{name}'")

    def build_request(self):
        """Build the final request dictionary."""
        return self.request

# Example usage
request = (DatasetRequest()
           .set_origin("cmorph")
           .set_region("quasi_global")
           .set_variable(["temperature", "precipitation"])
           .set_time_aggregation("daily")
           .set_experiment(["rcp8_5"]) 
           .build_request())

print(request)

{'origin': 'cmorph', 'region': 'quasi_global', 'variable': ['temperature', 'precipitation'], 'time_aggregation': 'daily', 'experiment': ['rcp8_5']}


### Tropospheric Humidity Dataset Summary

---

This dataset provides **monthly and zonally averaged tropospheric humidity profiles** derived from **GPS radio occultation (RO) measurements** from EUMETSAT’s Metop satellites. Humidity is critical to the Earth’s climate system, affecting the greenhouse effect, energy transport, cloud formation, and precipitation patterns.

#### **Key Features:**
- **Data Source:** EUMETSAT Radio Occultation Meteorology Satellite Application Facility (ROM SAF).
- **Types of Data:**
  - **Climate Data Record (CDR):** December 2006 - December 2016.
  - **Interim Climate Data Record (ICDR):** January 2017 - present.
- **Variables:**
  - **Specific Humidity:** Measured in g/kg, representing the mass of water vapor in air.
  - **Associated Variables:** Standard deviation, observation counts, sampling errors, model information fraction.
  
#### **Spatial and Temporal Coverage:**
- **Horizontal Resolution:** 5° latitude bands (global zonal means).
- **Vertical Resolution:** 0-12 km altitude (troposphere), with 0.2 km vertical intervals.
- **Temporal Coverage:** From 2006 to present.
- **Temporal Resolution:** Monthly.
  
#### **Data Format:**
- **File Format:** NetCDF-3.
- **Update Frequency:** Quarterly updates.

---

This dataset offers high vertical resolution humidity profiles, which are not influenced by cloud cover or surface type, making it ideal for climate analysis.

In [38]:
humidity = "satellite-humidity-profiles"

humidity_request = (DatasetRequest()
           .set_variable("all")
           .set_year(["2023"])
           .set_product_type("radio_occultation_data")
           .set_month(["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"])
           .build_request())
print(humidity_request)

humidity_file = "humidity-data.nc"

client.retrieve(humidity, humidity_request, humidity_file)




2024-10-03 12:39:21,819 INFO Request ID is b902cd5c-48de-489d-bc9b-34b738261e3a
2024-10-03 12:39:21,840 INFO status has been updated to accepted


{'variable': 'all', 'year': ['2023'], 'product_type': 'radio_occultation_data', 'month': ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']}


2024-10-03 12:39:23,376 INFO status has been updated to running
2024-10-03 12:39:29,049 INFO status has been updated to successful


'humidity-data.nc'

### Temperature and Precipitation

---

This dataset provides **high-resolution gridded temperature and precipitation observations** from various sources, designed for analysis of weather extremes and climate trends. The data are available at both **daily and monthly frequencies** and are regularly updated to incorporate recent observations.

#### **Key Features:**
- **Data Sources:** GISTEMP, Berkeley Earth, CPC, CPC-CONUS, CHIRPS, IMERG, CMORPH, GPCC, CRU.
- **Variables:** 
  - **Precipitation:** Measured in mm/day or mm/month.
  - **Temperature:** Air temperature at 2 meters above the surface (°C).
  - **Temperature Anomaly:** Deviations from the 1950-1980 climatology (°C).
- **Horizontal Coverage:** Varies by model (global, quasi-global, Africa, CONUS).
- **Horizontal Resolution:** Varies (0.1° x 0.1°, 0.25° x 0.25°, etc.).
- **Temporal Resolution:** Daily and monthly.
- **File Format:** NetCDF-4.
- **Projection:** Regular latitude-longitude grid.

#### **Applications:**
- **Climate Trend Analysis**
- **Weather Extremes**
- **Seasonal Drought Monitoring**
- **Rapid Response to Recent Phenomena**

#### **Data Sources:**
1. **GISTEMP-v4:** NASA's temperature analysis combining global historical data with sea surface temperatures.
2. **BERKEARTH:** Temperature records from 16 archives.
3. **CPC/CPC-CONUS:** Unified precipitation products using optimal interpolation.
4. **CHIRPS-v2:** Gridded rainfall time series for Africa.
5. **IMERG:** NASA’s satellite-based global precipitation estimates.
6. **CMORPH:** NOAA's precipitation estimates from microwave and geostationary IR data.
7. **GPCC:** Global precipitation climatology from long-term station records.
8. **CRU v4:** Gridded temperature and precipitation data based on station measurements.

#### **File Conventions:**
- **NetCDF-4** file format.
- **Metadata Conventions:** Climate and Forecast (CF) v1.6, ACDD v1.3.

---

This summary can be a quick reference to understand the scope and key features of the dataset.

In [4]:

dataset = "insitu-gridded-observations-global-and-regional"
request = (DatasetRequest()
           .set_origin("cmorph")
           .set_region("quasi_global")
           .set_variable(["temperature", "precipitation"])
           .set_time_aggregation("daily")
           .set_horizontal_aggregation(["0_5_x_0_5"])
           .set_year(["2020", "2021"])
           .set_version(["v1_0"])
           .build_request())
import json
print(json.dumps(request))

client = cdsapi.Client()
# client.retrieve(dataset, request).download()

{"origin": "cmorph", "region": "quasi_global", "variable": ["temperature", "precipitation"], "time_aggregation": "daily", "horizontal_aggregation": ["0_5_x_0_5"], "year": ["2020", "2021"], "version": ["v1_0"]}


NameError: name 'cdsapi' is not defined

# Temperature statistics for Europe derived from climate projections

In [32]:

request = (DatasetRequest()
           .set_variable("average_temperature")
           .set_period("year")
           .set_statistic(["time_average"])
           .set_experiment(["rcp8_5"])
           .set_ensemble_statistic(["ensemble_members_average", "ensemble_members_standard_deviation"])
           .build_request())

print(request)

dataset = "sis-temperature-statistics"

client.retrieve(dataset, request).download()

2024-10-03 12:32:05,681 INFO Request ID is e90777ac-8722-4ee8-9c6b-5e641155f1c0
2024-10-03 12:32:05,709 INFO status has been updated to accepted


{'variable': 'average_temperature', 'statistic': ['time_average'], 'experiment': ['rcp8_5'], 'ensemble_statistic': ['ensemble_members_average', 'ensemble_members_standard_deviation'], 'period': 'year'}
{'variable': 'average_temperature', 'statistic': ['time_average'], 'experiment': ['rcp8_5'], 'ensemble_statistic': ['ensemble_members_average', 'ensemble_members_standard_deviation'], 'period': 'year'}


2024-10-03 12:32:07,242 INFO status has been updated to running
2024-10-03 12:32:25,722 INFO status has been updated to successful


'86d7576f73ab1b831fd006740bd82a5f.zip'